In [1]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluation.new_structural_metric import symmetric_graph_set_distance,atom,cycle,neighborhood
from evaluate import evaluate
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
import sys
import json
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,get_graph_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess,get_generated_graph_data,_preprocess



# Rewired

In [2]:
import pandas as pd
def get_rewired_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/rewired/train1/{name}/{name}_rewired_with_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/rewired/train1/{name}/{name}_perturbed_with_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
   return  generated_graphs, np.array(targets)

In [8]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=['0','5','10','25','33','40','50','66','75','90','100']
metrics={}
for generator in ['rewired']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_rewired_graph_data(dataset_name+split,percent)
                
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.73925 0.8126 0.7620000000000001 0.8126
{'activations_time': 0.05635476112365723, 'fid': 0.053231914461093766, 'fid_time': 0.1711578369140625, 'kid': -0.004503846, 'kid_time': 5.511636734008789, 'precision': 0.96, 'recall': 0.96, 'f1_pr': 0.9600099999999999, 'precision_time': 0.8814888000488281, 'recall_time': 0.8814888000488281, 'f1_pr_time': 0.8814888000488281, 'density': 0.9465, 'coverage': 0.975, 'f1_dc': 0.9605486438860901, 'density_time': 0.41699695587158203, 'coverage_time': 0.41699695587158203, 'f1_dc_time': 0.41699695587158203, 'mmd_rbf': 0.00589260458946228, 'mmd_rbf_time': 0.7322475910186768, 'mmd_linear': 0.051

# Pertubed with ER

In [2]:

import pandas as pd
def get_corrupted_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/corrupted/train1/{name}/{name}_corrupted_with_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/corrupted/train1/{name}/{name}_perturbed_with_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
        map=nx.get_node_attributes(g, 'label')
        new_map={}
        for  k,val in map.items():
          new_map[k]=int(val)
          new_map
        nx.set_node_attributes(g,new_map,'label')
        map=nx.get_edge_attributes(g, 'label')
        new_map={}
        for  k,val in map.items():
          new_map[k]=int(val)
          new_map
        nx.set_edge_attributes(g,new_map,'label')
   return  generated_graphs, np.array(targets)

In [3]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn', 'structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in ['bbb_martins']:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

bbb_martins_50 corrupted
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
{'activations_time': 0.08747386932373047, 'fid': -7.119507472452824e-08, 'fid_time': 0.2391371726989746, 'kid': -0.098095536, 'kid_time': 6.203696966171265, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.00001, 'precision_time': 1.00077486038208, 'recall_time': 1.00077486038208, 'f1_pr_time': 1.00077486038208, 'density': 1.2008450704225353, 'coverage': 1.0, 'f1_dc': 1.0912682427575533, 'density_time': 0.8292548656463623, 'coverage_time': 0.8292548656463623, 'f1_dc_time': 0.8292548656463623, 'mmd_rbf': 4.76837158203125e-07, 'mmd_rbf_time': 1.1287791728973389, 'mmd_linear': 1.6268291e-13, 'mmd_linear_time': 0.08758187294006348, 'wl_mmd': 0.002816901408450704

In [9]:
train2_graphs[0].nodes(data=True)

NodeDataView({0: {'label': 7, 'attr': [6, 0, 1, 5, 0, 0, 0, 0, 0], 'label_name': 'N'}, 1: {'label': 6, 'attr': [5, 0, 2, 5, 0, 0, 0, 0, 0], 'label_name': 'C'}, 2: {'label': 6, 'attr': [5, 0, 4, 5, 2, 0, 2, 0, 0], 'label_name': 'C'}, 3: {'label': 7, 'attr': [6, 0, 3, 5, 1, 0, 1, 0, 0], 'label_name': 'N'}, 4: {'label': 6, 'attr': [5, 0, 3, 5, 0, 0, 1, 0, 0], 'label_name': 'C'}, 5: {'label': 8, 'attr': [7, 0, 1, 5, 0, 0, 1, 0, 0], 'label_name': 'O'}, 6: {'label': 6, 'attr': [5, 2, 4, 5, 1, 0, 2, 0, 1], 'label_name': 'C'}, 7: {'label': 6, 'attr': [5, 0, 4, 5, 2, 0, 2, 0, 1], 'label_name': 'C'}, 8: {'label': 6, 'attr': [5, 0, 4, 5, 2, 0, 2, 0, 1], 'label_name': 'C'}, 9: {'label': 6, 'attr': [5, 0, 4, 5, 2, 0, 2, 0, 1], 'label_name': 'C'}, 10: {'label': 6, 'attr': [5, 0, 4, 5, 2, 0, 2, 0, 1], 'label_name': 'C'}, 11: {'label': 6, 'attr': [5, 1, 4, 5, 1, 0, 2, 0, 1], 'label_name': 'C'}, 12: {'label': 6, 'attr': [5, 0, 3, 5, 0, 0, 1, 0, 0], 'label_name': 'C'}, 13: {'label': 8, 'attr': [7, 0, 1,

In [10]:
generated_graphs[0].nodes(data=True)

NodeDataView({0: {'label': 6, 'attr': tensor([5, 0, 3, 5, 1, 0, 1, 1, 1]), 'target': tensor(1)}, 1: {'label': 6, 'attr': tensor([5, 0, 4, 5, 2, 0, 2, 0, 1]), 'target': tensor(1)}, 2: {'label': 8, 'attr': tensor([7, 0, 2, 5, 0, 0, 1, 0, 0]), 'target': tensor(1)}, 3: {'label': 7, 'attr': tensor([5, 0, 4, 5, 2, 0, 2, 0, 1]), 'target': tensor(1)}, 4: {'label': 7, 'attr': tensor([6, 0, 3, 5, 0, 0, 1, 1, 1]), 'target': tensor(1)}, 5: {'label': 6, 'attr': tensor([5, 0, 4, 5, 2, 0, 2, 0, 1]), 'target': tensor(1)}, 6: {'label': 6, 'attr': tensor([7, 0, 2, 5, 1, 0, 2, 0, 0]), 'target': tensor(1)}, 7: {'label': 6, 'attr': tensor([5, 0, 3, 5, 1, 0, 1, 1, 1]), 'target': tensor(1)}, 8: {'label': 7, 'attr': tensor([5, 0, 4, 5, 2, 0, 2, 0, 0]), 'target': tensor(1)}, 9: {'label': 8, 'attr': tensor([5, 0, 3, 5, 0, 0, 1, 0, 0]), 'target': tensor(1)}, 10: {'label': 6, 'attr': tensor([5, 0, 3, 5, 1, 0, 1, 1, 1]), 'target': tensor(1)}, 11: {'label': 6, 'attr': tensor([5, 0, 3, 5, 1, 0, 1, 1, 1]), 'target': 

In [10]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn', 'structural', 'auc_roc']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue


ames_50 hiervae
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8856954315880887 0.9113656100738168 0.558198307318448 0.8816294154839988
{'activations_time': 0.23975300788879395, 'fid': -2.7648056288853695e-08, 'fid_time': 0.32007932662963867, 'kid': -0.033063095, 'kid_time': 0.31844210624694824, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.00001, 'precision_time': 2.3492929935455322, 'recall_time': 2.3492929935455322, 'f1_pr_time': 2.3492929935455322, 'density': 1.2002355712603063, 'coverage': 1.0, 'f1_dc': 1.0910165068038087, 'density_time': 1.5396878719329834, 'coverage_time': 1.5396878719329834, 'f1_dc_time': 1.5396878719329834, 'mmd_rbf': 9.5367431640625e-07, 'mmd_rbf_time': 2.605508804321289, 'mmd_linear': 4.420398e-13, 'mmd_linear_time': 0.2398831844329834, 'wl_mmd': 0.0007852375343541421, 'wl_mmd_time': 3.028226137161255, 'nspdk_mmd': 1.6653345369377348e-16, 'nspdk_mmd_time': 15.527252912521362, 'degree_mmd'

In [4]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_corrupted_graph_data(dataset_name+split,int(percent*100))
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
{'activations_time': 0.05511808395385742, 'fid': 0.13514366454909066, 'fid_time': 0.17839932441711426, 'kid': -0.0019218922, 'kid_time': 5.474117040634155, 'precision': 0.9425, 'recall': 0.8175, 'f1_pr': 0.8755711299871591, 'precision_time': 0.8653111457824707, 'recall_time': 0.8653111457824707, 'f1_pr_time': 0.8653111457824707, 'density': 0.684, 'coverage': 0.6925, 'f1_dc': 0.6882337562839624, 'density_time': 0.7219481468200684, 'coverage_time': 0.7219481468200684, 'f1_dc_time': 0.7219481468200684, 'mmd_rbf': 0.02698385715484619, 'mmd_rbf_time': 0.8984780311584473, 'mmd_linear': 0.11221051, 'mmd_linear_time': 0.055335283279418945, 'wl_mmd': 0.005, 'wl_mmd_time':

In [ ]:
0.7620000000000001 0.8126 0.71085 0.7885000000000001


# Mode Dropping


In [ ]:
import pandas as pd
def get_mode_dropping_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/mode_dropping/train1/{name}/{name}_no_of_clusters_dropped_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/mode_dropping/train1/{name}/{name}_fraction_of_clusters_dropped_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
   return  generated_graphs, np.array(targets)

In [3]:
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=[i for i in range(0, 29)]
[int(r* 28  ) for r in r]



[0, 1, 2, 4, 7, 9, 12, 14, 18, 19, 21, 23, 25, 26, 28]

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(0, 29)]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)



In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(0, 33)]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_2_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8065000000000001 0.82805 0.81105 0.8306000000000001
{'activations_time': 0.04191279411315918, 'fid': 0.4673616323057246, 'fid_time': 0.06281471252441406, 'kid': 0.02029562, 'kid_time': 0.07401895523071289, 'precision': 0.965, 'recall': 0.955, 'f1_pr': 0.9599839586045978, 'precision_time': 0.13985681533813477, 'recall_time': 0.13985681533813477, 'f1_pr_time': 0.13985681533813477, 'density': 0.9595000000000001, 'coverage': 0.97, 'f1_dc': 0.9647314307185207, 'density_time': 0.13818073272705078, 'coverage_time': 0.13818073272705078, 'f1_dc_time': 0.13818073272705078, 'mmd_rbf': 0.008294671773910522, 'mmd_rbf_time': 0.1538848876953125, 'mmd_linear': 0.28981638, 'mmd_linear_time': 0.04195761680603027, 'wl_mmd': 0.005, 'wl_mmd_time': 0.46073079109191895, 'nspdk_mmd': 0.003759602999053646, 'nspdk_mmd_time': 8.975366115570068, 'degree_mmd': 6.0964231288318516e-0

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn', 'structural', 'auc_roc']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 hiervae
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Cannot compute the classifier based metrics
Now computing structural based metrics


# Mode Collapse

In [ ]:




import pandas as pd
def get_mode_collapse_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/mode_collapse/train1/{name}/{name}_no_of_clusters_collapsed_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/mode_collapse/train1/{name}/{name}_fraction_of_clusters_collapsed_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
   return  generated_graphs, np.array(targets)

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(0, 32)]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7620000000000001 0.8126 0.7398 0.7906
{'activations_time': 0.04692196846008301, 'fid': 0.1320485407885741, 'fid_time': 0.05339813232421875, 'kid': -0.0021275282, 'kid_time': 0.06524181365966797, 'precision': 0.99, 'recall': 0.975, 'f1_pr': 0.9824527486743136, 'precision_time': 0.23654770851135254, 'recall_time': 0.23654770851135254, 'f1_pr_time': 0.23654770851135254, 'density': 1.0475, 'coverage': 0.98, 'f1_dc': 1.0126363982599431, 'density_time': 0.17254996299743652, 'coverage_time': 0.17254996299743652, 'f1_dc_time': 0.17254996299743652, 'mmd_rbf': 0.0057850182056427, 'mmd_rbf_time': 0.1793229579925537, 'mmd_linear': 0.0812089, 'mmd_linear_time': 0.04696917533874512, 'wl_mmd': 0.005, 'wl_mmd_time': 0.5076248645782471, 'nspdk_mmd': 0.004368843089490648, 'nspdk_mmd_time': 9.416892051696777, 'degree_mmd': 0.00012062901060772013, 'degree_mmd_time': 7.9832

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(0, 33)]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train2_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_2_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8065000000000001 0.82805 0.80315 0.8556500000000001
{'activations_time': 0.04502081871032715, 'fid': 0.5743247424700542, 'fid_time': 0.08963394165039062, 'kid': 0.01957488, 'kid_time': 0.06555962562561035, 'precision': 0.975, 'recall': 0.97, 'f1_pr': 0.9725035733308653, 'precision_time': 0.19227361679077148, 'recall_time': 0.19227361679077148, 'f1_pr_time': 0.19227361679077148, 'density': 1.066, 'coverage': 0.955, 'f1_dc': 1.0074617867215565, 'density_time': 0.11194682121276855, 'coverage_time': 0.11194682121276855, 'f1_dc_time': 0.11194682121276855, 'mmd_rbf': 0.005675438791513443, 'mmd_rbf_time': 0.14234685897827148, 'mmd_linear': 0.3699737, 'mmd_linear_time': 0.045076847076416016, 'wl_mmd': 0.005, 'wl_mmd_time': 0.4935731887817383, 'nspdk_mmd': 0.0040525563426479505, 'nspdk_mmd_time': 9.536576986312866, 'degree_mmd': 7.758755215681035e-05, 'degree_mm

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn', 'structural', 'auc_roc']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 hiervae
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Cannot compute the classifier based metrics
Now computing structural based metrics


# Additionally train1 versus train2

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 33)]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets= train2_graphs, train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            metrics['degree_of_perturbation']=percent
            print(metrics)

synthetic_1_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7620000000000001 0.8126 0.73925 0.8126
{'activations_time': 0.03923916816711426, 'fid': 0.04679155621931841, 'fid_time': 0.04503011703491211, 'kid': -0.0033990145, 'kid_time': 0.07478213310241699, 'precision': 0.975, 'recall': 0.9775, 'f1_pr': 0.9762583995042304, 'precision_time': 0.20879530906677246, 'recall_time': 0.20879530906677246, 'f1_pr_time': 0.20879530906677246, 'density': 0.9960000000000001, 'coverage': 0.9, 'f1_dc': 0.9455796458898114, 'density_time': 0.13438010215759277, 'coverage_time': 0.13438010215759277, 'f1_dc_time': 0.13438010215759277, 'mmd_rbf': 0.006395334377884865, 'mmd_rbf_time': 0.1645030975341797, 'mmd_linear': 0.03953388, 'mmd_linear_time': 0.0392911434173584, 'wl_mmd': 0.005, 'wl_mmd_time': 0.46033692359924316, 'nspdk_mmd': 0.004269677911791503, 'nspdk_mmd_time': 9.157461881637573, 'degree_mmd': 0.00014829539083982546, 'degree

# SWINGNN as we increase the number of training epochs

In [ ]:

import pandas as pd
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np
def get_swingnn_graph_data(name,percent,split,path='data/SWINGNN_experiments/'):
     path+=f'{percent}-percent/'
     path_postives=path+r'{}/{}/pos/*.pkl'.format(name,name+split)
     files = glob(path_postives)
     generated_pos_graphs=pd.read_pickle(files[0])
     path_negatives=path+r'{}/{}/neg/*.pkl'.format(name,name+split)
     files = glob(path_negatives)
     generated_neg_graphs=pd.read_pickle(files[0])
     generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)

     return  generated_graphs, np.array(generated_targets)

In [ ]:
name='synthetic_1'
percent='100'
split='_50'
get_swingnn_graph_data(name,percent,split)

([<networkx.classes.graph.Graph at 0x2a91acd30>,
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np

def get_swingnn_molecular_data(name,percent, path='data/SWINGNN_experiments/', generator_name='stgg',return_smiles=False):
     path+=f'{percent}-percent/'
     pos_list, neg_list=[],[]
     path_postives=path+r'{}/{}/pos/*.txt'.format(name,name+'_50')
     files = glob(path_postives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               pos_list.append(line.strip())

     path_negatives=path+r'{}/{}/neg/*.txt'.format(name,name+'_50')
     files = glob(path_negatives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               neg_list.append(line.strip())

     pos_graphs=list_of_smiles_to_nx_graphs(pos_list)
     neg_graphs=list_of_smiles_to_nx_graphs(neg_list)
     #print(len(neg_graphs))
     generated_graphs,generated_targets = pos_graphs + neg_graphs, [1]*len(pos_graphs)+[0]*len(neg_graphs)
     #generated_graphs, generated_targets = shuffle(generated_graphs, generated_targets)
     generated_graphs, generated_targets = remove_empty_graphs_and_targets(generated_graphs, generated_targets)
     if return_smiles:
          return  generated_graphs, generated_targets,pos_list+neg_list
     return  generated_graphs, np.array(generated_targets)

In [16]:
name='ames'
percent='100'
split='_50'
_,generated_targets, generated_smiles=get_swingnn_molecular_data(name,percent,return_smiles=True) 

In [10]:
generated_targets[-1]

0

## Molecular graphs

### nn and structural 

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_25','_33','_40','_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
             for percent in experiments:
                print(dataset_name+split)
                train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                generated_graphs, generated_targets=get_swingnn_molecular_data(dataset_name,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

ames_50
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.21366095542907715, 'fid': 26.50553763515596, 'fid_time': 0.21585965156555176, 'kid': 0.9191108, 'kid_time': 0.305006742477417, 'precision': 0.8629760502552022, 'recall': 0.5826462504907735, 'f1_pr': 0.6956412606466655, 'precision_time': 1.2940280437469482, 'recall_time': 1.2940280437469482, 'f1_pr_time': 1.2940280437469482, 'density': 0.7016882606988615, 'coverage': 0.3376521397722811, 'f1_dc': 0.4559283499249382, 'density_time': 0.938338041305542, 'coverage_time': 0.938338041305542, 'f1_dc_time': 0.938338041305542, 'mmd_rbf': 0.18399828672409058, 'mmd_rbf_time': 1.4124457836151123, 'mmd_linear': 16.260548, 'mmd_linear_time': 0.2137889862060547, 'wl_mmd': 0.0007852375343541421, 'wl_mmd_time': 2.1692988872528076, 'nspdk_mmd': 0.08389942961717695, 'nspdk_mmd_time': 12.742777824401855, 'degree_mmd': 0.0228826161270963, 'degree_mmd_time': 37.297131, 'cluster_mmd': 0.0003904567516506

#### train2

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            print(dataset_name+split)
            train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
            generated_graphs, generated_targets=train2_graphs , train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_25
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.09668684005737305, 'fid': 2.9564222480168496, 'fid_time': 0.1282970905303955, 'kid': 0.01800251, 'kid_time': 0.15801072120666504, 'precision': 0.969363707776905, 'recall': 0.9630793401413983, 'f1_pr': 0.9662213055809473, 'precision_time': 0.5455377101898193, 'recall_time': 0.5455377101898193, 'f1_pr_time': 0.5455377101898193, 'density': 1.0227808326787118, 'coverage': 0.9748625294579732, 'f1_dc': 0.9982569686750845, 'density_time': 0.399242639541626, 'coverage_time': 0.399242639541626, 'f1_dc_time': 0.399242639541626, 'mmd_rbf': 0.002381354570388794, 'mmd_rbf_time': 0.5534439086914062, 'mmd_linear': 0.16101262, 'mmd_linear_time': 0.09678387641906738, 'wl_mmd': 0.0015710919088766694, 'wl_mmd_time': 3.6933791637420654, 'nspdk_mmd': 0.0014485484440663887, 'nspdk_mmd_time': 7.593865871429443, 'degree_mmd': 0.00026359444890333705, 'degree_mmd_time': 28.933659, 'cluster_mmd': 7.422813

### molecular

In [8]:
from evaluation.moses.metrics.metrics import get_all_metrics
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split1='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn','gdss']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            for percent in experiments:
                print(dataset_name+split1)
                graphs, splits =get_data(dataset_name+split1, return_smiles=True)
                train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                _,generated_targets, generated_smiles=get_swingnn_molecular_data(dataset_name,percent,return_smiles=True) 
                metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                metrics['dataset']=dataset_name+split1
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100 
                print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.992, 'unique@10000': 0.9752650176678446, 'FCD/Test': 17.196146057508543, 'SNN/Test': 0.27051011988074813, 'Frag/Test': 0.1531511401333041, 'Scaf/Test': 0.05274522872358611, 'FCD/TestSF': 37.4445580256326, 'SNN/TestSF': 0.19734603975408443, 'Frag/TestSF': 0.27915602040639564, 'Scaf/TestSF': 4.1041894040527716e-05, 'IntDiv': 0.9190337370237776, 'IntDiv2': 0.9022144090456522, 'Filters': 0.5127601099332548, 'logP': 0.19126302709069487, 'SA': 1.4890440133925626, 'QED': 0.08059893151943091, 'weight': 54.34200942285044, 'Novelty': 0.9967793880837359, 'dataset': 'ames_50', 'generator_name': 'swingnn', 'numer_of_epochs_trained': 250.0}
ames_50
{'valid': 1.0, 'unique@1000': 0.987, 'unique@10000': 0.9733019238319591, 'FCD/Test': 15.621304290928059, 'SNN/Test': 0.28040955265129036, 'Frag/Test': 0.19670744762450332, 'Scaf/Test': 0.07717364804524973, 'FCD/TestSF': 35.64221637716807, 'SNN/TestSF': 0.21063697174583457, 'Frag/TestSF': 0.4119607673591432, 'Scaf/Te

#### train1


In [9]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train1_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9996073812328229, 'FCD/Test': 0.8156919603444166, 'SNN/Test': 0.575357792673038, 'Frag/Test': 0.9878820451873492, 'Scaf/Test': 0.8593618010840082, 'FCD/TestSF': 22.270570217406373, 'SNN/TestSF': 0.33544746274439363, 'Frag/TestSF': -0.06553137100764106, 'Scaf/TestSF': 0.13905682249091533, 'IntDiv': 0.9105206593714188, 'IntDiv2': 0.8919648544524341, 'Filters': 0.5551629367883785, 'logP': 0.08376508833922258, 'SA': 0.04752492131394497, 'QED': 0.0039007607322941876, 'weight': 4.032424028268554, 'Novelty': 0.997643362136685, 'dataset': 'ames_50', 'generator_name': 'train1'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9943661971830986, 'unique@10000': 0.9943661971830986, 'FCD/Test': 3.6841985920171965, 'SNN/Test': 0.5081955164146256, 'Frag/Test': 0.966294915338154, 'Scaf/Test': 0.5502519513284054, 'FCD/TestSF': 19.44386748202205, 'SNN/TestSF': 0.3730856233761764, 'Frag/TestSF': -0.10547896046759009, 'Scaf/TestSF': 0.0249430999

In [10]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train2_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9988221436984688, 'FCD/Test': 2.7711166694643907e-13, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': 22.184285088577205, 'SNN/TestSF': 0.33677946461981123, 'Frag/TestSF': -0.02581333437055111, 'Scaf/TestSF': 0.20333785426573803, 'IntDiv': 0.9102053951183938, 'IntDiv2': 0.8921645781772736, 'Filters': 0.5618374558303887, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.0, 'Novelty': 0.00039308176100628933, 'dataset': 'ames_50', 'generator_name': 'train2'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9957805907172996, 'unique@10000': 0.9957805907172996, 'FCD/Test': 9.947598300641403e-14, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': 20.86298113850406, 'SNN/TestSF': 0.3738190557404242, 'Frag/TestSF': 0.06172323226684595, 'Scaf/TestSF': 0.02450489523326016, 'IntDiv': 0.8879810418473504, 'IntDiv2': 0.8712773076208551, 'Filters': 0.7848101265822784, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.

In [11]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1/2+train2/2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            half=len(train1_smiles)//2
            train3_smiles=train1_smiles[:half] + train2_smiles[:half]
            metrics=get_all_metrics(gen=train3_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9972505891594659, 'FCD/Test': 1.4296307269705935, 'SNN/Test': 0.80414747127214, 'Frag/Test': 0.9769171525419632, 'Scaf/Test': 0.9058604214675904, 'FCD/TestSF': 24.652601465116575, 'SNN/TestSF': 0.3295319009622016, 'Frag/TestSF': -0.014604840934317576, 'Scaf/TestSF': 0.13555299909832097, 'IntDiv': 0.9005626242852879, 'IntDiv2': 0.8800491646635825, 'Filters': 0.4615082482325216, 'logP': 0.13850204932346513, 'SA': 0.06508225753225652, 'QED': 0.02354588307221548, 'weight': 11.541142909838637, 'Novelty': 0.4990153603781016, 'dataset': 'ames_50', 'generator_name': 'train1/2+train2/2'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9929577464788732, 'unique@10000': 0.9929577464788732, 'FCD/Test': 3.722576671925424, 'SNN/Test': 0.74866054233321, 'Frag/Test': 0.9741186915938969, 'Scaf/Test': 0.7575674931332002, 'FCD/TestSF': 19.48522451637382, 'SNN/TestSF': 0.378976951418838, 'Frag/TestSF': 0.21368685111121732, 'Scaf/TestSF': 0.0076

## Synthetic graphs 

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['swingnn']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['swingnn']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in experiments:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_swingnn_graph_data(dataset_name,percent,split)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

synthetic_1_25
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7019 0.7620000000000001 0.42635 0.5492999999999999
{'activations_time': 0.021010875701904297, 'fid': 1992.0837783679606, 'fid_time': 0.04033160209655762, 'kid': 32062.93, 'kid_time': 4.84860897064209, 'precision': 0.01485148514851485, 'recall': 0.025, 'f1_pr': 0.018644188556302177, 'precision_time': 0.17827486991882324, 'recall_time': 0.17827486991882324, 'f1_pr_time': 0.17827486991882324, 'density': 0.0029702970297029703, 'coverage': 0.005, 'f1_dc': 0.0037373549602240336, 'density_time': 0.08560895919799805, 'coverage_time': 0.08560895919799805, 'f1_dc_time': 0.08560895919799805, 'mmd_rbf': 1.2035624831914902, 'mmd_rbf_t

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['train2']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets=train2_graphs,train2_targets
            splits_for_auc_roc=[train2_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)


synthetic_1_25
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7019 0.7620000000000001 0.7061000000000001 0.7620000000000001
{'activations_time': 0.09806990623474121, 'fid': 0.22037899701926023, 'fid_time': 0.32657504081726074, 'kid': -0.002235651, 'kid_time': 0.1197969913482666, 'precision': 0.97, 'recall': 0.99, 'f1_pr': 0.9799079602248955, 'precision_time': 1.095655918121338, 'recall_time': 1.095655918121338, 'f1_pr_time': 1.095655918121338, 'density': 1.026, 'coverage': 0.98, 'f1_dc': 1.0024825875115901, 'density_time': 0.22144103050231934, 'coverage_time': 0.22144103050231934, 'f1_dc_time': 0.22144103050231934, 'mmd_rbf': 0.00850243540480733, 'mmd_rbf_time': 0.36985182762145996, 'mmd_linear': 0.03589476, 'mmd_linear_time': 0.098114013671875, 'wl_mmd': 0.01, 'wl_mmd_time': 0.24774694442749023, 'nspdk_mmd': 0.008445549078421655, 'nspdk_mmd_time': 4.805341005325317, 'degree_mmd': 0.00026336114711855707, 'degree_m